# Tutorial 03 — MNIST Sliding Window + Temporal Ensemble

**Improvement over sliding window**: Take output every 20 steps and ensemble (mean).

## Ensemble Strategy

```
Step 20  → Output₁ ──┐
Step 40  → Output₂ ──┼── Mean → Final Prediction
Step 60  → Output₃ ──┤
Step 80  → Output₄ ──┤
Step 100 → Output₅ ──┤
Step 101 → Output₆ ──┘
```

## Why This Helps

- Different timesteps capture different integration states
- Averaging reduces variance
- Mistakes at one timestep can be corrected by others

In [ ]:
import os
import sys
from pathlib import Path

notebook_dir = Path.cwd()
for parent in [notebook_dir] + list(notebook_dir.parents):
    candidate = parent / "src"
    if (candidate / "soen_toolkit").exists():
        sys.path.insert(0, str(candidate))
        break

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import gzip
import urllib.request
import struct
from tqdm import tqdm

torch.set_float32_matmul_precision('high')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"PyTorch: {torch.__version__}")
print(f"Device: {device}")

## 1. Hyperparameters

In [ ]:
# ============================================================
# KEY HYPERPARAMETERS
# ============================================================

# Sliding window
WINDOW_SIZE = 8
N_ROW_STEPS = 20
N_COL_STEPS = 20
STEPS_PER_SWEEP = N_ROW_STEPS + N_COL_STEPS  # 40

# Timing
N_INPUT_STEPS = 100
N_SETTLE_STEPS = 1

# ENSEMBLE: Take output every ENSEMBLE_INTERVAL steps
ENSEMBLE_INTERVAL = 20
# This gives outputs at steps: 20, 40, 60, 80, 100, 101

# Network
HIDDEN_DIM = 28
INPUT_DIM = WINDOW_SIZE
OUTPUT_DIM = 10

# SOEN dynamics
DT = 0.1
GAMMA_PLUS = 0.1
GAMMA_MINUS = 0.01

# Training
BATCH_SIZE = 128
EPOCHS = 30
LR = 0.005

# Calculate ensemble steps
ensemble_steps = list(range(ENSEMBLE_INTERVAL, N_INPUT_STEPS + 1, ENSEMBLE_INTERVAL))
ensemble_steps.append(N_INPUT_STEPS + N_SETTLE_STEPS)  # Add final settle step
print(f"Ensemble at steps: {ensemble_steps}")
print(f"Number of ensemble members: {len(ensemble_steps)}")

## 2. Load MNIST

In [ ]:
def download_mnist_file(filename, base_url="https://ossci-datasets.s3.amazonaws.com/mnist/"):
    data_dir = Path("./data/mnist")
    data_dir.mkdir(parents=True, exist_ok=True)
    filepath = data_dir / filename
    if not filepath.exists():
        print(f"Downloading {filename}...")
        urllib.request.urlretrieve(base_url + filename, filepath)
    return filepath

def read_mnist_images(filepath):
    with gzip.open(filepath, 'rb') as f:
        magic, num, rows, cols = struct.unpack('>IIII', f.read(16))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(num, rows, cols)

def read_mnist_labels(filepath):
    with gzip.open(filepath, 'rb') as f:
        magic, num = struct.unpack('>II', f.read(8))
        return np.frombuffer(f.read(), dtype=np.uint8)

def load_mnist():
    train_img = read_mnist_images(download_mnist_file("train-images-idx3-ubyte.gz")).astype(np.float32) / 255.0
    train_lbl = read_mnist_labels(download_mnist_file("train-labels-idx1-ubyte.gz")).astype(np.int64)
    test_img = read_mnist_images(download_mnist_file("t10k-images-idx3-ubyte.gz")).astype(np.float32) / 255.0
    test_lbl = read_mnist_labels(download_mnist_file("t10k-labels-idx1-ubyte.gz")).astype(np.int64)
    
    np.random.seed(42)
    idx = np.random.permutation(len(train_img))
    n_val = 6000
    
    val_img, val_lbl = train_img[idx[:n_val]], train_lbl[idx[:n_val]]
    train_img, train_lbl = train_img[idx[n_val:]], train_lbl[idx[n_val:]]
    
    print(f"Train: {train_img.shape}, Val: {val_img.shape}, Test: {test_img.shape}")
    return (train_img, train_lbl), (val_img, val_lbl), (test_img, test_lbl)

(train_data, train_labels), (val_data, val_labels), (test_data, test_labels) = load_mnist()

## 3. Visualize Ensemble Strategy

In [ ]:
def visualize_ensemble_strategy():
    """Visualize which timesteps are used for ensemble."""
    fig, ax = plt.subplots(figsize=(14, 4))
    
    total_steps = N_INPUT_STEPS + N_SETTLE_STEPS
    
    # Draw sweeps
    for sweep in range(3):
        start = sweep * STEPS_PER_SWEEP
        if start >= N_INPUT_STEPS:
            break
        
        # Row phase
        row_end = min(start + N_ROW_STEPS, N_INPUT_STEPS)
        if row_end > start:
            ax.barh(0, row_end - start, left=start, height=0.4, 
                    color='blue', alpha=0.5, edgecolor='black')
        
        # Column phase
        col_start = start + N_ROW_STEPS
        col_end = min(col_start + N_COL_STEPS, N_INPUT_STEPS)
        if col_end > col_start:
            ax.barh(0, col_end - col_start, left=col_start, height=0.4,
                    color='orange', alpha=0.5, edgecolor='black')
    
    # Settle phase
    ax.barh(0, N_SETTLE_STEPS, left=N_INPUT_STEPS, height=0.4,
            color='green', alpha=0.5, edgecolor='black')
    
    # Mark ensemble steps
    for i, step in enumerate(ensemble_steps):
        ax.axvline(x=step - 0.5, color='red', linewidth=2, linestyle='--')
        ax.scatter([step - 0.5], [0], color='red', s=150, zorder=5, marker='v')
        ax.text(step - 0.5, 0.35, f'E{i+1}\n(step {step})', ha='center', fontsize=8, 
                color='red', fontweight='bold')
    
    ax.set_xlim(-1, total_steps + 2)
    ax.set_ylim(-0.5, 0.6)
    ax.set_xlabel('Timestep')
    ax.set_title(f'Temporal Ensemble: Average outputs from {len(ensemble_steps)} timesteps')
    ax.set_yticks([])
    
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='blue', alpha=0.5, label='Row phase'),
        Patch(facecolor='orange', alpha=0.5, label='Col phase'),
        Patch(facecolor='green', alpha=0.5, label='Settle'),
        Patch(facecolor='red', label='Ensemble output'),
    ]
    ax.legend(handles=legend_elements, loc='upper right')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nEnsemble strategy:")
    print(f"  • Take output every {ENSEMBLE_INTERVAL} steps")
    print(f"  • Ensemble steps: {ensemble_steps}")
    print(f"  • Final prediction = mean of {len(ensemble_steps)} outputs")

visualize_ensemble_strategy()

## 4. Sliding Window SOEN with Temporal Ensemble

In [ ]:
class SlidingWindowEnsembleSOEN(nn.Module):
    """
    SOEN model with sliding window input and temporal ensemble.
    
    Takes output every `ensemble_interval` steps and averages for final prediction.
    """
    
    def __init__(self, hidden_dim=28, window_size=8, output_dim=10,
                 n_row_steps=20, n_col_steps=20,
                 n_input_steps=100, n_settle_steps=1,
                 ensemble_interval=20,
                 dt=0.1, gamma_plus=0.1, gamma_minus=0.01):
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.window_size = window_size
        self.output_dim = output_dim
        self.n_row_steps = n_row_steps
        self.n_col_steps = n_col_steps
        self.steps_per_sweep = n_row_steps + n_col_steps
        self.n_input_steps = n_input_steps
        self.n_settle_steps = n_settle_steps
        self.ensemble_interval = ensemble_interval
        self.dt = dt
        self.gamma_plus = gamma_plus
        self.gamma_minus = gamma_minus
        
        # Pre-compute ensemble steps (1-indexed)
        self.ensemble_steps = list(range(ensemble_interval, n_input_steps + 1, ensemble_interval))
        self.ensemble_steps.append(n_input_steps + n_settle_steps)
        
        # Weights
        self.W_i2h = nn.Parameter(torch.empty(hidden_dim, window_size))
        self.W_h2h = nn.Parameter(torch.empty(hidden_dim, hidden_dim))
        self.W_h2o = nn.Parameter(torch.empty(output_dim, hidden_dim))
        self.bias_h = nn.Parameter(torch.zeros(hidden_dim))
        self.bias_o = nn.Parameter(torch.zeros(output_dim))
        
        self._init_weights()
    
    def _init_weights(self):
        nn.init.uniform_(self.W_i2h, -0.2, 0.2)
        nn.init.normal_(self.W_h2h, 0, 0.1)
        nn.init.normal_(self.W_h2o, 0, 0.2)
        with torch.no_grad():
            self.W_h2h.fill_diagonal_(0)
    
    def source_function(self, phi):
        return torch.sigmoid(5 * phi)
    
    def get_window_input(self, images, step):
        """Extract 8-pixel window for each neuron."""
        step_in_sweep = step % self.steps_per_sweep
        
        if step_in_sweep < self.n_row_steps:
            window_start = step_in_sweep
            window_end = min(window_start + self.window_size, 28)
            window_start = window_end - self.window_size
            window = images[:, :, window_start:window_end]
        else:
            col_step = step_in_sweep - self.n_row_steps
            window_start = col_step
            window_end = min(window_start + self.window_size, 28)
            window_start = window_end - self.window_size
            window = images[:, window_start:window_end, :].transpose(1, 2)
        
        return window
    
    def step(self, s, window_input=None):
        """Single timestep update."""
        if window_input is not None:
            input_contrib = (window_input * self.W_i2h.unsqueeze(0)).sum(dim=2)
        else:
            input_contrib = 0
        
        recurrent_contrib = F.linear(s, self.W_h2h)
        phi = input_contrib + recurrent_contrib + self.bias_h
        
        g = self.source_function(phi)
        dsdt = self.gamma_plus * g - self.gamma_minus * s
        s_new = s + self.dt * dsdt
        
        return s_new
    
    def forward(self, images, return_all=False):
        """
        Forward pass with temporal ensemble.
        
        Args:
            images: (batch, 28, 28)
            return_all: If True, return all individual outputs
        
        Returns:
            output: Ensembled output (batch, 10)
            states: Dict with intermediate states
        """
        batch_size = images.shape[0]
        s = torch.zeros(batch_size, self.hidden_dim, device=images.device)
        
        all_states = []
        all_outputs = []
        ensemble_outputs = []  # Outputs at ensemble steps
        
        current_step = 0
        
        # INPUT PHASE
        for t in range(self.n_input_steps):
            window = self.get_window_input(images, t)
            s = self.step(s, window)
            current_step += 1
            
            output = F.linear(s, self.W_h2o, self.bias_o)
            all_outputs.append(output)
            
            # Check if this is an ensemble step
            if current_step in self.ensemble_steps:
                ensemble_outputs.append(output)
            
            if return_all:
                all_states.append(s.clone())
        
        # SETTLE PHASE
        for t in range(self.n_settle_steps):
            s = self.step(s, window_input=None)
            current_step += 1
            
            output = F.linear(s, self.W_h2o, self.bias_o)
            all_outputs.append(output)
            
            if current_step in self.ensemble_steps:
                ensemble_outputs.append(output)
            
            if return_all:
                all_states.append(s.clone())
        
        # ENSEMBLE: Average outputs from all ensemble steps
        ensemble_stack = torch.stack(ensemble_outputs, dim=0)  # (n_ensemble, batch, 10)
        ensembled_output = ensemble_stack.mean(dim=0)  # (batch, 10)
        
        return ensembled_output, {
            'all_outputs': all_outputs,
            'ensemble_outputs': ensemble_outputs,
            'ensemble_steps': self.ensemble_steps,
            'all_states': all_states if return_all else None,
            'final_state': s
        }

# Create model
model = SlidingWindowEnsembleSOEN(
    hidden_dim=HIDDEN_DIM,
    window_size=WINDOW_SIZE,
    output_dim=OUTPUT_DIM,
    n_row_steps=N_ROW_STEPS,
    n_col_steps=N_COL_STEPS,
    n_input_steps=N_INPUT_STEPS,
    n_settle_steps=N_SETTLE_STEPS,
    ensemble_interval=ENSEMBLE_INTERVAL,
    dt=DT,
    gamma_plus=GAMMA_PLUS,
    gamma_minus=GAMMA_MINUS
).to(device)

print(f"Model created with temporal ensemble")
print(f"  Ensemble steps: {model.ensemble_steps}")
print(f"  Number of ensemble members: {len(model.ensemble_steps)}")
print(f"  Parameters: {sum(p.numel() for p in model.parameters())}")

## 5. Training

In [ ]:
def train_model(model, train_data, train_labels, val_data, val_labels,
                epochs=30, batch_size=128, lr=0.005):
    """
    Train the ensemble SOEN model.
    """
    train_dataset = TensorDataset(
        torch.tensor(train_data, dtype=torch.float32),
        torch.tensor(train_labels, dtype=torch.long)
    )
    val_dataset = TensorDataset(
        torch.tensor(val_data, dtype=torch.float32),
        torch.tensor(val_labels, dtype=torch.long)
    )
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    best_val_acc = 0
    best_state = None
    
    print("="*60)
    print("SLIDING WINDOW + TEMPORAL ENSEMBLE TRAINING")
    print("="*60)
    print(f"Ensemble steps: {model.ensemble_steps}")
    print(f"Ensemble members: {len(model.ensemble_steps)}")
    print("="*60)
    
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        epoch_correct = 0
        epoch_total = 0
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        for x, labels in pbar:
            x, labels = x.to(device), labels.to(device)
            
            optimizer.zero_grad()
            output, _ = model(x)
            loss = F.cross_entropy(output, labels)
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            
            with torch.no_grad():
                model.W_h2h.fill_diagonal_(0)
            
            pred = output.argmax(dim=1)
            epoch_correct += (pred == labels).sum().item()
            epoch_total += len(labels)
            epoch_loss += loss.item() * len(labels)
            
            pbar.set_postfix({'loss': f'{loss.item():.4f}', 'acc': f'{epoch_correct/epoch_total:.3f}'})
        
        scheduler.step()
        
        train_loss = epoch_loss / epoch_total
        train_acc = epoch_correct / epoch_total
        
        # Validation
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for x, labels in val_loader:
                x, labels = x.to(device), labels.to(device)
                output, _ = model(x)
                loss = F.cross_entropy(output, labels)
                val_loss += loss.item() * len(labels)
                val_correct += (output.argmax(dim=1) == labels).sum().item()
                val_total += len(labels)
        
        val_loss /= val_total
        val_acc = val_correct / val_total
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_state = {k: v.clone() for k, v in model.state_dict().items()}
        
        print(f"Epoch {epoch+1}: train_loss={train_loss:.4f}, train_acc={train_acc:.3f}, "
              f"val_loss={val_loss:.4f}, val_acc={val_acc:.3f} {'*' if val_acc == best_val_acc else ''}")
    
    if best_state:
        model.load_state_dict(best_state)
    print(f"\nBest validation accuracy: {best_val_acc:.4f}")
    
    return history

history = train_model(model, train_data, train_labels, val_data, val_labels,
                      epochs=EPOCHS, batch_size=BATCH_SIZE, lr=LR)

## 6. Visualize Training

In [ ]:
def plot_training(history):
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    axes[0].plot(history['train_loss'], label='Train')
    axes[0].plot(history['val_loss'], label='Val')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Loss (Sliding Window + Ensemble)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(history['train_acc'], label='Train')
    axes[1].plot(history['val_acc'], label='Val')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_title('Accuracy (Sliding Window + Ensemble)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_training(history)

## 7. Evaluate

In [ ]:
@torch.no_grad()
def evaluate(model, test_data, test_labels):
    model.eval()
    
    test_dataset = TensorDataset(
        torch.tensor(test_data, dtype=torch.float32),
        torch.tensor(test_labels, dtype=torch.long)
    )
    test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)
    
    all_preds = []
    all_labels = []
    
    for x, labels in tqdm(test_loader, desc="Testing"):
        x = x.to(device)
        output, _ = model(x)
        all_preds.append(output.argmax(dim=1).cpu())
        all_labels.append(labels)
    
    all_preds = torch.cat(all_preds)
    all_labels = torch.cat(all_labels)
    accuracy = (all_preds == all_labels).float().mean().item()
    
    print(f"\n{'='*60}")
    print(f"TEST ACCURACY (Ensemble): {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"{'='*60}")
    
    return accuracy

test_acc = evaluate(model, test_data, test_labels)

## 8. Compare Ensemble vs Individual Steps

In [ ]:
@torch.no_grad()
def compare_ensemble_vs_individual(model, test_data, test_labels, max_samples=2000):
    """
    Compare ensemble accuracy vs individual timestep accuracy.
    """
    model.eval()
    
    x = torch.tensor(test_data[:max_samples], dtype=torch.float32).to(device)
    labels = torch.tensor(test_labels[:max_samples], dtype=torch.long)
    
    ensembled_output, states = model(x)
    ensemble_outputs = states['ensemble_outputs']
    
    # Individual accuracies
    individual_accs = []
    for i, (step, output) in enumerate(zip(model.ensemble_steps, ensemble_outputs)):
        pred = output.argmax(dim=1).cpu()
        acc = (pred == labels).float().mean().item()
        individual_accs.append((step, acc))
        print(f"Step {step:3d}: {acc:.4f}")
    
    # Ensemble accuracy
    ensemble_pred = ensembled_output.argmax(dim=1).cpu()
    ensemble_acc = (ensemble_pred == labels).float().mean().item()
    print(f"\nENSEMBLE: {ensemble_acc:.4f}")
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 5))
    
    steps = [s for s, a in individual_accs]
    accs = [a for s, a in individual_accs]
    
    ax.bar(range(len(steps)), accs, color='blue', alpha=0.7, label='Individual')
    ax.axhline(y=ensemble_acc, color='red', linewidth=2, linestyle='--', 
               label=f'Ensemble: {ensemble_acc:.4f}')
    
    ax.set_xticks(range(len(steps)))
    ax.set_xticklabels([f'Step {s}' for s in steps], rotation=45, ha='right')
    ax.set_ylabel('Accuracy')
    ax.set_title('Ensemble vs Individual Timestep Accuracy')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    # Improvement stats
    best_individual = max(accs)
    mean_individual = np.mean(accs)
    print(f"\nComparison:")
    print(f"  Best individual: {best_individual:.4f}")
    print(f"  Mean individual: {mean_individual:.4f}")
    print(f"  Ensemble:        {ensemble_acc:.4f}")
    print(f"  Improvement over best: {(ensemble_acc - best_individual)*100:+.2f}%")
    print(f"  Improvement over mean: {(ensemble_acc - mean_individual)*100:+.2f}%")

compare_ensemble_vs_individual(model, test_data, test_labels)

## 9. Visualize Ensemble Agreement

In [ ]:
@torch.no_grad()
def visualize_ensemble_agreement(model, image, label):
    """
    Visualize how individual ensemble members vote.
    """
    model.eval()
    x = torch.tensor(image, dtype=torch.float32).unsqueeze(0).to(device)
    
    ensembled_output, states = model(x)
    ensemble_outputs = states['ensemble_outputs']
    
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    
    # Original image
    axes[0, 0].imshow(image, cmap='gray')
    axes[0, 0].set_title(f'Input (Label: {label})')
    axes[0, 0].axis('off')
    
    # Individual predictions
    all_probs = []
    for i, (step, output) in enumerate(zip(model.ensemble_steps[:6], ensemble_outputs[:6])):
        if i >= 6:
            break
        row, col = divmod(i + 1, 4)
        if i + 1 < 4:
            row, col = 0, i + 1
        else:
            row, col = 1, i + 1 - 4
        
        probs = F.softmax(output, dim=1).squeeze().cpu().numpy()
        all_probs.append(probs)
        pred = probs.argmax()
        
        colors = ['green' if j == label else 'blue' for j in range(10)]
        colors[pred] = 'red' if pred != label else 'green'
        
        axes[row, col].bar(range(10), probs, color=colors, alpha=0.7)
        axes[row, col].set_title(f'Step {step}: pred={pred} {"✓" if pred == label else "✗"}', 
                                  fontsize=10)
        axes[row, col].set_xticks(range(10))
        axes[row, col].set_ylim(0, 1)
    
    # Ensemble result
    axes[1, 3].clear()
    ensemble_probs = F.softmax(ensembled_output, dim=1).squeeze().cpu().numpy()
    ensemble_pred = ensemble_probs.argmax()
    colors = ['green' if j == label else 'blue' for j in range(10)]
    colors[ensemble_pred] = 'red' if ensemble_pred != label else 'green'
    
    axes[1, 3].bar(range(10), ensemble_probs, color=colors, alpha=0.9, edgecolor='black', linewidth=2)
    axes[1, 3].set_title(f'ENSEMBLE: pred={ensemble_pred} {"✓" if ensemble_pred == label else "✗"}',
                          fontsize=12, fontweight='bold')
    axes[1, 3].set_xticks(range(10))
    axes[1, 3].set_ylim(0, 1)
    
    plt.suptitle(f'Temporal Ensemble Voting (True label: {label})', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Print vote summary
    print("\nIndividual predictions:")
    for i, (step, output) in enumerate(zip(model.ensemble_steps, ensemble_outputs)):
        pred = output.argmax(dim=1).item()
        print(f"  Step {step:3d}: {pred} {'✓' if pred == label else '✗'}")
    print(f"\n  ENSEMBLE: {ensemble_pred} {'✓' if ensemble_pred == label else '✗'}")

# Visualize a few samples
for i in range(3):
    visualize_ensemble_agreement(model, test_data[i], test_labels[i])

## Summary

| Aspect | Single Output | Temporal Ensemble |
|--------|--------------|-------------------|
| Output source | Step 101 only | Steps 20, 40, 60, 80, 100, 101 |
| Variance | Higher | **Lower** |
| Robustness | Less | **More** |

### Why Ensemble Helps

1. **Different integration states**: Each timestep captures different features
2. **Error averaging**: Mistakes at one step can be corrected by others
3. **Redundancy**: Row and column phases both contribute
4. **No extra parameters**: Same model, just different outputs